In [ ]:
import numpy as np
import pandas as pd

import sys
import os

from PIL import Image, ImageFont, ImageDraw
from PIL.ImageChops import add, subtract, multiply, difference, screen

import cv2

import matplotlib.pyplot as plt

# Pillow

In [ ]:
im_tom = Image.open("../test_images/tomatoes.jpeg")
im_tom = im_tom.resize((200,200))
im_tom

In [ ]:
im_fl = Image.open("../test_images/flour.jpg")
im_fl = im_fl.resize((200,200))
im_fl

In [ ]:
bg1 = Image.open("../test_images/background_fridge.jpg")
bg1 = bg1.resize((200,200))
bg1

In [ ]:
bg2 = Image.open("../test_images/background_table.jpg")
#bg2 = tf.image.resize(bg2,[200,200])
bg2 = bg2.resize((200,200))
bg2

In [ ]:
im_apple = Image.open("../test_images/apple.jpeg")
im_apple = im_apple.resize((300,200))
im_apple

# OpenCV

## Image Overlays Using Bitwise Operations in OpenCV

In [ ]:
# Load two images - background and ingredient 
# TODO: pipeline for images
background = cv2.imread('../test_images/background_table.jpg')
rows_b, cols_b, channels_b = background.shape

ingredient = cv2.imread('../test_images/banana.jpg')

# resize ingredient image to make sure it is always smaller than the background
# keep the aspect ratio by resizing https://stackoverflow.com/questions/44650888/resize-an-image-without-distortion-opencv
aspect_ratio = float(ingredient.shape[1])/float(ingredient.shape[0])
rows_i = int(rows_b/2)
cols_i = int(rows_i/aspect_ratio)
dim_res = (rows_i, cols_i)
ingredient = cv2.resize(ingredient, dim_res)


# Create a Region of Images (=ROI) in a randomised matter 
# roi = Region Of Images https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_core/py_basic_ops/py_basic_ops.html
rows, cols, channels = ingredient.shape

# TODO position on picture needs to be randomised
#start_rows = np.random.randint(0, size=rows)
#start_cols = np.random.randint(0, size=cols)
#roi = background[start_rows:rows, start_cols:cols] 
roi = background[0:rows, 0:cols]



# Now create a mask of the ingredient and create its inverse mask also
img2gray = cv2.cvtColor(ingredient, cv2.COLOR_BGR2GRAY)

# everything higher than the threshold (i.e. lighter), will become black
# the rest (i.e. the darker pixels filled by the object), will become white
# with THRESH_OTSU the threshold is adjustable according to the image
# similar to the idea to take the mean value between black and white as a threshold
# the GaussianBluring smooths the image to make the black & white more precise
blur = cv2.GaussianBlur(img2gray, (5,5), 0)
ret, mask = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
mask_inv = cv2.bitwise_not(mask)


# Black-out the area of the ingredient in ROI in the background image
background_bg = cv2.bitwise_and(roi, roi, mask = mask_inv)

# Take only region of the ingredient from the ingredient image.
ingredient_fg = cv2.bitwise_and(ingredient, ingredient, mask = mask)

# Put the ingredient in ROI and modify the main image
dst = cv2.add(background_bg, ingredient_fg)

#TODO positions need to be randomised (see above)
#background[start_rows:rows, start_cols:cols] = dst
background[0:rows, 0:cols] = dst

cv2.imshow('res', background)
#cv2.imshow('img2gray',img2gray)
#cv2.imshow('mask', mask)
#cv2.imshow('mask_inv', mask_inv)
#cv2.imshow('ret', ret)
#cv2.imshow('background_bg', background_bg)
#cv2.imshow('ingredient_fg', ingredient_fg)
#cv2.imshow('dst', dst)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)